In [1]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import seaborn as sns
from PIL import Image
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr
from time import time

In [2]:
from inrnet import args as args_module
from inrnet import inn, experiments, optim, util, losses, models, jobs as job_mgmt
from inrnet.data import dataloader
from inrnet.experiments import depth
import inrnet.inn.functional as inrF
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

# interactive

### segment

In [ ]:
from inrnet.inn.nets import wgan
wgan.translate_wgan_model()

In [ ]:
python train.py -j=seg_nn -c=seg_nn

In [ ]:
python train.py -j=inet_scr_mlp -c=inet_scr_mlp
bash train.sh inet_scr_mlp inet_scr_mlp

In [ ]:
python train.py -j=seg_inn -c=seg_train
bash train.sh seg_nn seg_nn
bash train.sh seg_scr seg_scr
bash train.sh seg_inn seg_train

### classify

In [ ]:
python train.py -j=inet_inn_train -c=inet_train

In [3]:
from inrnet.experiments import classify
classify.analyze_logit_mismatch()

In [ ]:
bash train.sh inet_nn inet_nn
bash train.sh inet_inn_tr inet_train
bash train.sh inet_scr inet_scr
bash train.sh inet_con inet_cont
bash train.sh inet_mlp inet_mlp
bash train.sh inet_lg inet_lg
bash train.sh inet_lg_mlp inet_lg_mlp

bash infer.sh i1x_nn inet_val inet_nn
bash infer.sh i2x_nn in2x_val inet_nn
bash infer.sh ihx_nn inhx_val inet_nn
bash infer.sh i1x_inn inet_val inet_inn_tr
bash infer.sh i2x_inn in2x_val inet_inn_tr
bash infer.sh ihx_inn inhx_val inet_inn_tr
bash infer.sh i1x_scr inet_val inet_scr
bash infer.sh i2x_scr in2x_val inet_scr
bash infer.sh ihx_scr inhx_val inet_scr
bash infer.sh i1x_mlp inet_val inet_mlp
bash infer.sh i2x_mlp in2x_val inet_mlp
bash infer.sh ihx_mlp inhx_val inet_mlp
bash infer.sh i1x_fast inet_val inet_fast
bash infer.sh i2x_fast in2x_val inet_fast
bash infer.sh ihx_fast inhx_val inet_fast
bash infer.sh i1x_con inet_val inet_con
bash infer.sh i2x_con in2x_val inet_con
bash infer.sh ihx_con inhx_val inet_con
python infer.py -j=inet_inn_val -c=inet_val -t=inet_inn_train

In [ ]:
bash infer.sh i1x_inn inet_val inet_inn_tr2x
bash infer.sh i1x_scr inet_val inet_scr_tr2x
bash infer.sh i2x_inn in2x_val inet_inn_tr2x
bash infer.sh i2x_scr in2x_val inet_scr_tr2x
bash infer.sh ihx_inn inhx_val inet_inn_tr2x
bash infer.sh ihx_scr inhx_val inet_scr_tr2x

### Generate

In [ ]:
python train.py -j=gen_inn -c=gen_train
bash train.sh gen_inn gen_train

In [ ]:
python train.py -j=gen_nn -c=gen_nn
bash train.sh gen_nn gen_nn

### random fourier features

In [4]:
import urllib.request

import torch

import torch.nn as nn
import tqdm

import numpy as np
import cv2

import cv2
import imageio
import torch
import numpy as np
from tqdm.notebook import tqdm as tqdm
from torch import nn
import matplotlib.pyplot as plt

def tensor_to_numpy(tensor: torch.Tensor) -> np.ndarray:
    tensor = tensor * 256
    tensor[tensor > 255] = 255
    tensor[tensor < 0] = 0
    tensor = tensor.type(torch.uint8).permute(1, 2, 0).cpu().numpy()

    return tensor

def get_image():
    image_url = 'https://live.staticflickr.com/7492/15677707699_d9d67acf9d_b.jpg'
    img = imageio.imread(image_url)[..., :3] / 255.
    c = [img.shape[0] // 2, img.shape[1] // 2]
    r = 256
    img = img[c[0] - r:c[0] + r, c[1] - r:c[1] + r]

    return img

### etc

In [3]:
import torch
import rff

X = torch.randn((256, 256, 2))
encoding = rff.layers.GaussianEncoding(sigma=10.0, input_size=2, encoded_size=256)
Xp = encoding(X)

In [215]:
sd = torch.load('/data/vision/polina/users/clintonw/code/diffcoord/temp/wgan.pth')['state_dict']

In [181]:
base = torchvision.models.efficientnet_b0(pretrained=False, device='cuda')
optim = torch.optim.Adam(base.parameters())
optim.zero_grad()

In [ ]:
CUDA_VISIBLE_DEVICES=3 python train.py -j=inet2 -c=inet

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [97]:
base = torch.load(osp.expanduser('~/code/diffcoord/temp/upernet_convnext.pth'))

In [203]:
img = plt.imread('/data/vision/polina/scratch/clintonw/datasets/cityscapes/gtCoarse/val/frankfurt/frankfurt_000001_080091_gtCoarse_labelIds.png')

In [ ]:
base = torchvision.models.efficientnet_b0(pretrained=True)
base

In [42]:
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"
ds = torchvision.datasets.CIFAR10(root=DS_DIR, transform=transforms.ToTensor())

In [78]:
torch.cuda.empty_cache()
coords = util.meshgrid_coords(64,64)
#spacing = torch.tensor((4/127,4/127), device="cuda")
values = torch.randn(coords.size(0),6, device=coords.device)

In [146]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/diffcoord/inrnet")
query_string = "inn.layers.reshape"
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

/data/vision/polina/users/clintonw/code/diffcoord/inrnet/inn/nets/wgan.py


## sympy

In [151]:
from sympy import sin, cos, tan, exp, log, integrate
from sympy.abc import a,b,c,n,m,x,y
from sympy import Point, Polygon
from sympy.integrals.intpoly import polytope_integrate

In [149]:
from sympy.functions.special import polynomials, spherical_harmonics

In [173]:
function = (3*polynomials.legendre(2, x)-2*polynomials.legendre(1, x) - 1) * \
    (.6*polynomials.legendre(4, y)-polynomials.legendre(1, y)+1.1)

In [ ]:
function

In [162]:
polygon = Polygon(Point(0, 0), Point(0, 1), Point(1, 1), Point(1, 0))
# polys = [1, x, y, x*y, x**2*y, x*y**2]
# expr = x*y

In [174]:
polytope_integrate(polygon, expr=function)

nan

In [146]:
polytope_integrate(polygon, expr=expr)

1/4

In [147]:
polytope_integrate(polygon, polys, max_degree=3)

{1: 1, x: 1/2, y: 1/2, x*y: 1/4, x**2*y: 1/6, x*y**2: 1/6}

## inet12

In [3]:
from inrnet.models.inrs import siren
keys = siren.get_siren_keys()

In [16]:
from robustness.tools.imagenet_helpers import ImageNetHierarchy, common_superclass_wnid
in_path = '/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch'
in_hier = ImageNetHierarchy(in_path, in_path)
superclass_wnid = common_superclass_wnid('big_12')
class_ranges, label_map = in_hier.get_subclasses(superclass_wnid,
                                                 balanced=True)

In [10]:
subpaths = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/train.txt",
            "r").read().split('\n')
labels = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/labels.txt",
            "r").read().split('\n')
labels = [l[:l.find(',')] for l in labels[:-1]]
class_labels = [np.array(labels)[list(cr)].tolist() for cr in class_ranges]

In [ ]:
sub_to_super = {}
for supercls,subclasses in enumerate(class_ranges):
    for subcls in subclasses:
        sub_to_super[subcls] = supercls

# big12path = f"{DS_DIR}/inrnet/big_12.pkl"
# pickle.dump((label_map, class_ranges, sub_to_super), open(big12path, 'wb'))
# label_map, class_ranges, sub_to_super = pickle.load(open(big12path, 'rb'))

In [ ]:
label_to_super = {}
for supercls,subclasses in enumerate(class_labels):
    for subcls in subclasses:
        label_to_super[subcls] = supercls

In [ ]:
N = 800
subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

for p in subpaths_by_cls:
    np.random.shuffle(p)

split_paths_by_cls = {'train':[p[:N] for p in subpaths_by_cls],
                     'test':[p[N:] for p in subpaths_by_cls]}

# big12path = f"{DS_DIR}/inrnet/big_12_labels.pkl"
# pickle.dump((split_paths_by_cls, class_labels, label_to_super), open(big12path, 'wb'))
# split_paths_by_cls, class_labels, label_to_super = pickle.load(open(big12path, 'rb'))

In [19]:
subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

for p in subpaths_by_cls:
    np.random.shuffle(p)

In [26]:
N = 23400
subpaths_by_cls = [p[:N] for p in subpaths_by_cls]

In [27]:
big12path = f"{DS_DIR}/inrnet/big_12_extra.pkl"
pickle.dump(subpaths_by_cls, open(big12path, 'wb'))
# subpaths_by_cls = pickle.load(open(big12path, 'rb'))

In [ ]:
paths = util.glob2('/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/train_*.pt')
for p in paths:
    weights, seg = torch.load(p)
    break

# Tables

### Classify

In [11]:
RESULTS_DIR = osp.expanduser("~/code/diffcoord/results")

model_jobs = {'EffNet-T':('i1x_nn', 'i2x_nn', 'ihx_nn'),
#     'INR-tuned (ours)':('i1x_inn', 'i2x_inn', 'ihx_inn'),
    'INR-scratch (ours)':('i1x_scr', 'i2x_scr', 'ihx_scr'),
    'INR-MLP':('i1x_mlp', 'i2x_mlp', 'ihx_mlp'),
     }
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

EffNet-T & 67.5\% & 60.4\% & 48.7\% \\
INR-scratch (ours) & 52.1\% & 51.2\% & 24.4\% \\
missing i1x_mlp results at /data/vision/polina/users/clintonw/code/diffcoord/results/i1x_mlp/stats.pt
missing i2x_mlp results at /data/vision/polina/users/clintonw/code/diffcoord/results/i2x_mlp/stats.pt
missing ihx_mlp results at /data/vision/polina/users/clintonw/code/diffcoord/results/ihx_mlp/stats.pt
INR-MLP & nan & nan & nan \\
INR-fast & 39.8\% & 35.8\% & 23.9\% \\


In [ ]:
jobs = ["raw_m_rgae", "dit_m_rgae", "raw_m_caae", "dit_m_caae",
        "raw_m_ipg", "dit_m_ipg", "raw_m_star", "dms_1"]
df = tables.get_results_table()
subtable = df.loc[jobs]

def latex_format(value, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$'%n_decimals).format(value)        
    else:
        return (r'${0:.%df}$'%n_decimals).format(value)
def plus_minus_format(mean, std, n_decimals=2, bold=False):
    if bold:
        return (r'$\bm{{{0:.%df}}}$\scriptsize$\bm{{\pm {1:.%df}}}$'%(n_decimals, n_decimals)).format(mean, std)        
    else:
        return (r'${0:.%df}$\scriptsize$\pm {1:.%df}$'%(n_decimals, n_decimals)).format(mean, std)

def print_row(name, row, bold=()):
    field1 = latex_format(row["FID_tuned"], bold=1 in bold)
    field2 = latex_format(row["F_{1/8}"], bold=2 in bold)
    field3 = latex_format(row["F_8"], bold=3 in bold)
    field4 = plus_minus_format(row["age mean error"], row["age error STD"], bold=4 in bold)
    print(" & ".join([name, field1, field2, field3, field4]) + r" \\")

### Segment

In [ ]:
model_jobs = {'EffNet-T':('c1x_nn', 'c2x_nn', 'chx_nn'),
    'INR-tuned (ours)':('c1x_inn', 'c2x_inn', 'chx_inn'),
    'INR-scratch (ours)':('c1x_scr', 'c2x_scr', 'chx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

### Generate

In [ ]:
model_jobs = {'EffNet-T':('f1x_nn', 'f2x_nn', 'fhx_nn'),
    'INR-tuned (ours)':('f1x_inn', 'f2x_inn', 'fhx_inn'),
    'INR-scratch (ours)':('f1x_scr', 'f2x_scr', 'fhx_scr')}
N = 200*12
for model,row in model_jobs.items():
    rowstr = [model]
    for job in row:
        path = osp.join(RESULTS_DIR, job, "stats.pt")
        if not osp.exists(path):
            print(f'missing {job} results at {path}')
            rowstr.append("nan")
        else:
            top1, top3 = torch.load(path)
            rowstr.append(util.format_float(top3/N*100, n_decimals=1)+r"\%")
    print(' & '.join(map(str,rowstr)) + r' \\')

# Figs

## for framework

In [ ]:
xy = util.generate_quasirandom_sequence(d=2, n=256)
plt.rcParams["figure.figsize"] = (2,1)
fig,ax = plt.subplots()
ax.scatter(xy[:,0], xy[:,1], s=2, c='k')
ax.plot();

In [ ]:
kernel = model.features[0][0].weight[3,1].detach()
plt.rcParams["figure.figsize"] = (.5,.5)
fig,ax = plt.subplots()
print(kernel.min().item(), kernel.max().item())
ax.imshow(kernel, vmin=-2.9, vmax=2.9);
plt.axis('off');

In [100]:
from scipy.interpolate import RectBivariateSpline as Spline2D
K = [2.5,2.5]
h,w=3,3
bbox = (-K[0]/2, K[0]/2, -K[1]/2, K[1]/2)
x,y = (np.linspace(bbox[0]/h*(h-1), bbox[1]/h*(h-1), h),
       np.linspace(bbox[2]/w*(w-1), bbox[3]/w*(w-1), w))
bs = Spline2D(x,y, kernel, bbox=bbox, kx=2,ky=2, s=0)
tx,ty,c = [torch.tensor(z).float() for z in bs.tck]
c = c.reshape(h,w)

In [106]:
H = 50
xy = util.meshgrid_coords(H,H).cpu()
w_oi = []
X = xy[:,0].unsqueeze(1)
Y = xy[:,1].unsqueeze(1)
px = py = 2

values, kx = (tx<=X).min(dim=-1)
values, ky = (ty<=Y).min(dim=-1)
kx -= 1
ky -= 1
kx[values] = tx.size(-1)-px-2
ky[values] = ty.size(-1)-py-2

Ctrl = c.view(1, *c.shape[-2:])
for z in range(X.size(0)):
    D = Ctrl[:, kx[z]-px : kx[z]+1, ky[z]-py : ky[z]+1].clone()

    for r in range(1, px + 1):
        try:
            alphax = (X[z,0] - tx[kx[z]-px+1:kx[z]+1]) / (
                tx[2+kx[z]-r:2+kx[z]-r+px] - tx[kx[z]-px+1:kx[z]+1])
        except RuntimeError:
            print("input off the grid")
        for j in range(px, r - 1, -1):
            D[:,j] = (1-alphax[j-1]) * D[:,j-1] + alphax[j-1] * D[:,j].clone()

    for r in range(1, py + 1):
        alphay = (Y[z,0] - ty[ky[z]-py+1:ky[z]+1]) / (
            ty[2+ky[z]-r:2+ky[z]-r+py] - ty[ky[z]-py+1:ky[z]+1])
        for j in range(py, r-1, -1):
            D[:,px,j] = (1-alphay[j-1]) * D[:,px,j-1].clone() + alphay[j-1] * D[:,px,j].clone()

    w_oi.append(D[:,px,py])

w = torch.stack(w_oi).view(xy.size(0))

In [107]:
k = w.reshape(H,H)

In [ ]:
fig,ax = plt.subplots()
print(k.min().item(), k.max().item())
ax.imshow(k, vmin=-2.9, vmax=2.9);
plt.axis('off');

## analysis

In [ ]:
from inrnet.experiments import classify
classify.analyze_change_resolution_grid_vs_qmc()

In [ ]:
ax = sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.5,.8,10)+torch.randn(10)/8, label="qmc");
sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.1,.8,10)+torch.randn(10)/8, label="grid", ax=ax);
ax.set_xlabel("Number of sample points");
ax.set_ylabel("ImageNet top-1 error");

### Deviation over depth

### Resolution vs. output (grid & QMC)

In [201]:
base_logits, grid_logits, grid_masks, qmc_logits, qmc_masks = torch.load(osp.expanduser(
    '~/code/diffcoord/temp/change_resolution_grid_vs_qmc.pt'))

In [212]:
grid_dists = (torch.cat(grid_logits, dim=0) - base_logits.cpu()).norm(dim=-1)
qmc_dists = (torch.cat(qmc_logits, dim=0) - base_logits.cpu()).norm(dim=-1)

In [ ]:
torch.sort(base_logits[0]).indices

In [ ]:
torch.sort(qmc_logits[-1][0]).indices

In [ ]:
plt.bar(grid_dists)

In [214]:
qmc_dists

tensor([15575.2344,  1041.2139,   471.6435,   106.5965,    59.1255,    45.8519,
           41.1897,    39.6904])

In [208]:
front_g_m = torch.stack([m[0].float().mean() for m in grid_masks], dim=0)
front_g_s = torch.stack([m[0].float().std() for m in grid_masks], dim=0)
back_g_m = torch.stack([m[1].float().mean() for m in grid_masks], dim=0)
back_g_s = torch.stack([m[1].float().std() for m in grid_masks], dim=0)

In [209]:
front_g_m, front_g_s

(tensor([  1.0000,  10.5625,   8.6289,  15.0156,  46.4102,  60.5479, 160.5764,
         196.0000]),
 tensor([ 0.0000,  3.0957,  1.0093,  4.7626,  6.1273, 22.8689, 20.5887, 87.4629]))

In [210]:
back_g_m, back_g_s

(tensor([  1.0000,   2.2500,   7.5625,   7.5625,  39.0625,  31.6406, 136.5977,
         142.5039]),
 tensor([ 0.0000,  1.1255,  1.7078,  1.7078,  9.7238,  6.8699, 33.1658, 35.0628]))

In [185]:
front_q_m = torch.stack([m[0].float().mean() for m in qmc_masks], dim=0)
front_q_s = torch.stack([m[0].float().std() for m in qmc_masks], dim=0)
back_q_m = torch.stack([m[1].float().mean() for m in qmc_masks], dim=0)
back_q_s = torch.stack([m[1].float().std() for m in qmc_masks], dim=0)

In [190]:
front_q_m, front_q_s

(tensor([  1.8906,   4.7576,   9.3281,  18.5356,  37.3281,  74.0309, 146.2527,
         292.7811]),
 tensor([ 0.6695,  1.0991,  1.6431,  2.6684,  4.9519,  9.4219, 18.3863, 36.7924]))

In [191]:
back_q_m, back_q_s

(tensor([  3.2500,   4.6970,   8.4375,  15.3492,  31.3125,  63.8085, 126.1699,
         250.4597]),
 tensor([ 1.3416,  1.3803,  2.1958,  3.7912,  7.4199, 15.3288, 30.1571, 59.7511]))

### RQMC variance

In [ ]:
base_logits, logits, masks = torch.load(osp.expanduser('~/code/diffcoord/temp/output_variance_rqmc.pt'))

In [160]:
logit_dists = (torch.cat(logits, dim=0) - base_logits.cpu()).norm(dim=-1)

In [ ]:
logit_dists

In [ ]:
fronts.mean(1), fronts.std(1), fronts.amin(1), fronts.amax(1)

In [ ]:
backs.mean(1), backs.std(1), backs.amin(1), backs.amax(1)

In [157]:
fronts = torch.stack([m[0] for m in masks], dim=0).float()
backs = torch.stack([m[1] for m in masks], dim=0).float()

### grid-QMC interpolation

In [ ]:
base_logits, grid_logits, grid_mask, qmc_logits, qmc_mask, intermediate_logits, intermediate_masks = torch.load(osp.expanduser('~/code/diffcoord/temp/analyze_logit_mismatch.pt'))

x = np.arange(0,1.01,.05)
fronts = torch.stack([grid_mask[0], *[m[0] for m in intermediate_masks[::-1]], qmc_mask[0]], dim=0).float()
backs = torch.stack([grid_mask[1], *[m[1] for m in intermediate_masks[::-1]], qmc_mask[1]], dim=0).float()
f_m, f_s = fronts.mean(1), fronts.std(1)
b_m, b_s = backs.mean(1), backs.std(1)

fig,ax = plt.subplots()
width = .007
ax.errorbar(x-width/2, f_m, yerr=f_s, label='front')
ax.errorbar(x+width/2, b_m, yerr=b_s, label='back')

In [ ]:
y = (torch.cat([grid_logits, *intermediate_logits[::-1], qmc_logits], dim=0) - base_logits.cpu()).norm(dim=1)
plt.plot(x,y)

# slurm

In [45]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334
inet_len = 50000
in_tr_len = 50000

In [ ]:
configs/convnext/upernet_convnext_tiny
configs/_base_/models/upernet_convnext.py

In [ ]:
for ix in range(0,places_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/places/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"place{ix//800}", "fit_places", ix)

In [ ]:
for ix in range(0,in_tr_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_train/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_in_tr", ix)

In [86]:
for ix in range(0,12000,100):
    c,i=ix//1000, ix%1000
    if i >= 800:
        i-=800
        s = 'test'
    else:
        s='train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{c}/{s}_{i+3}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

In [148]:
#23400 * 12 = 280800 total images
#15000/12 + 800 = 2050 current images per class
for ix in range(24000,36000,100):
    cls, start_ix = ix % 12, ix // 12 + 800
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{cls}/train_{start_ix+4}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

sh fit_inr.sh inet250 fit_i12 25000
sh fit_inr.sh inet251 fit_i12 25100
sh fit_inr.sh inet252 fit_i12 25200
sh fit_inr.sh inet253 fit_i12 25300
sh fit_inr.sh inet254 fit_i12 25400
sh fit_inr.sh inet255 fit_i12 25500
sh fit_inr.sh inet256 fit_i12 25600
sh fit_inr.sh inet257 fit_i12 25700
sh fit_inr.sh inet258 fit_i12 25800
sh fit_inr.sh inet259 fit_i12 25900
sh fit_inr.sh inet260 fit_i12 26000
sh fit_inr.sh inet261 fit_i12 26100
sh fit_inr.sh inet262 fit_i12 26200
sh fit_inr.sh inet263 fit_i12 26300
sh fit_inr.sh inet264 fit_i12 26400
sh fit_inr.sh inet265 fit_i12 26500
sh fit_inr.sh inet266 fit_i12 26600
sh fit_inr.sh inet267 fit_i12 26700
sh fit_inr.sh inet268 fit_i12 26800
sh fit_inr.sh inet269 fit_i12 26900
sh fit_inr.sh inet270 fit_i12 27000
sh fit_inr.sh inet271 fit_i12 27100
sh fit_inr.sh inet272 fit_i12 27200
sh fit_inr.sh inet273 fit_i12 27300
sh fit_inr.sh inet274 fit_i12 27400
sh fit_inr.sh inet275 fit_i12 27500
sh fit_inr.sh inet276 fit_i12 27600
sh fit_inr.sh inet277 fit_i1

In [ ]:
for ix in range(0,8189,100):
    i = ix
    if i < 1020:
        s='train'
    elif i < 2040:
        s='val'
        i -= 1020
    else:
        s='test'
        i -= 2040
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/flowers/{s}_{i+50}.pt"):
        print("sh fit_inr.sh", f"flow{ix//100}", "fit_flower", ix)

In [ ]:
for ix in range(0,3475,100):
    i = ix
    if i >= 2975:
        i -= 2975
        s = 'val'
    else:
        s = 'train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes_nonorm/{s}_{i+30}.pt"):
        print("sh fit_inr.sh", f"city{ix//100}", "fit_city", ix)

In [46]:
for ix in range(0,inet_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_val/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_inet", ix)

In [37]:
x[0,0] = 1

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [ ]:
for ix in range(0,10000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_test/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cit{ix//5000}", "fit_ciftest", ix)
for ix in range(0,50000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_train/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cif{ix//5000}", "fit_cifar", ix)

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/1/loss_*")

In [ ]:
paths = util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/0/loss_train_10*")
for p in paths[10:100]:
    print(open(p,'r').read())